In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Set root package root directory.
import os
os.environ['ML_FOR_SG_ROOT'] = 'Z:\Dev\Github\ml-for-sg'

# Create shotgun data manager.
from lib.source_generator.shotgun.source_generator import ShotgunSourceGenerator
shotgun_source_generator = ShotgunSourceGenerator()

In [4]:
raw_data = shotgun_source_generator.get_raw_data(70, 3)

In [5]:
shot_data = shotgun_source_generator.reformat_raw_shot_data(raw_data)

In [6]:
shot_data

{'bunny_010_0010': {'cost': 19200,
  'feature_dict': {'text_features': {'description': u'alice,be,beginning,case,down,fall,get,in,it,likely,more,over,than,the,through,to,true,very,was,which,would',
    'notes': u'about,added,adoption,afraid,after,again,age,air,alarm,alice,all,alone,always,am,among,an,and,angry,another,answer,any,anything,apple,are,arm,as,at,attending,away,back,bathing,be,beasts,because,beg,began,behind,best,bit,branches,bright,brightened,bringing,broken,but,butterfly,by,cake,came,can,cant,cat,cats,changed,changing,cheated,children,chin,chorus,close,coaxing,come,comfort,coming,consultation,conversation,corner,could,course,cried,cry,curled,dear,delight,delighted,did,dinah,dipped,directly,do,dodo,doesn,dogs,don,door,down,drew,drink,drop,dropped,dry,duck,ears,eat,eats,eggs,else,em,enough,ever,every,everything,excellent,eyed,eyes,face,family,fan,far,fat,fear,find,finished,fire,first,fond,foot,for,forgot,found,four,from,garden,general,get,girl,glass,gloves,go,going,got,grave

In [7]:
value_columns = shotgun_source_generator.generate_value_column_data(shot_data)

In [8]:
value_columns

defaultdict(list,
            {'assets': [u'alice',
              u'backdrop',
              u'banning',
              u'billowy',
              u'bunny',
              u'bus',
              u'car',
              u'caterpillar',
              u'clouds',
              u'crow',
              u'darcy',
              u'deserted',
              u'fern',
              u'fields',
              u'flash',
              u'flower',
              u'forest',
              u'grass',
              u'highway',
              u'int',
              u'island',
              u'jimmy',
              u'launcher',
              u'leaf',
              u'mansion',
              u'matte',
              u'morning',
              u'mouse',
              u'mrs',
              u'police',
              u'queen',
              u'scare',
              u'scary',
              u'school',
              u'shack',
              u'sky',
              u'squirrel',
              u'stormy',
              u'street',
            

In [9]:
text_columns = shotgun_source_generator.generate_text_column_data(shot_data)

In [10]:
text_columns

[u'about',
 u'above',
 u'added',
 u'adding',
 u'adoption',
 u'afraid',
 u'after',
 u'again',
 u'against',
 u'age',
 u'air',
 u'alarm',
 u'alice',
 u'alices',
 u'all',
 u'allow',
 u'almost',
 u'alone',
 u'along',
 u'altoget',
 u'altogether',
 u'always',
 u'am',
 u'among',
 u'an',
 u'and',
 u'angry',
 u'another',
 u'answer',
 u'anxiously',
 u'any',
 u'anything',
 u'appeared',
 u'apple',
 u'are',
 u'arm',
 u'arms',
 u'as',
 u'asking',
 u'at',
 u'ate',
 u'attending',
 u'avoid',
 u'away',
 u'back',
 u'bad',
 u'bark',
 u'bathing',
 u'be',
 u'beasts',
 u'beautifully',
 u'because',
 u'bed',
 u'bee',
 u'before',
 u'beg',
 u'began',
 u'beginning',
 u'behind',
 u'being',
 u'bend',
 u'best',
 u'better',
 u'bill',
 u'bills',
 u'bit',
 u'blow',
 u'bones',
 u'book',
 u'books',
 u'bottle',
 u'branches',
 u'bright',
 u'brightened',
 u'bring',
 u'bringing',
 u'broken',
 u'busy',
 u'but',
 u'buttercup',
 u'butterfly',
 u'by',
 u'cake',
 u'cakes',
 u'called',
 u'came',
 u'can',
 u'candle',
 u'cant',
 u'ca

In [11]:
df_y_full = shotgun_source_generator.generate_cost_data(shot_data)

In [12]:
heatmap = []
df_x_full = None
text_feature_index = []
value_feature_index = []
#text_feature_values = []
#print value_columns
for _, shot_values in shot_data.iteritems():
    for column, _ in shot_values['feature_dict']['value_features'].iteritems():
        if value_columns[column] is None:
            value_feature_index.append(column)
        else:
            for value in value_columns[column]:
                value_feature_index.append('{}__{}'.format(column, value))

    #text_feature_values = value['feature_dict']['text_features']
    #value_feature_values = value['feature_dict']['value_features']
            
value_feature_index = sorted(set(value_feature_index))
text_feature_index = sorted(set(text_columns))

df_x_values = []
df_x_text_values = []
value_feature_values = []
text_feature_values = []
cost_values = []

for shot_name, shot_datum in shot_data.iteritems():
    value_feature_values = []
    cost_values.append(shot_datum['cost'])
    for value_feature in value_feature_index:
        if '__' in value_feature:
            column = value_feature.split('__')[0]
            sub_column = value_feature.split('__')[1]
            value_feature_values.append(True if sub_column in shot_datum['feature_dict']['value_features'][column] else False)
        else:
            value_feature_values.append(shot_datum['feature_dict']['value_features'][value_feature])
    df_x_values.append(value_feature_values)
for shot_name, shot_datum in shot_data.iteritems():
    text_feature_values = []
    for text_feature in text_feature_index:
        result = False
        for key, value in shot_datum['feature_dict']['text_features'].iteritems():
            if text_feature in value:
                result = True
                break
        text_feature_values.append(result)
    df_x_text_values.append(text_feature_values)
print value_feature_index
print df_x_values
df_x = pd.DataFrame(df_x_values, index=shot_data.keys(), columns=value_feature_index)
df_y = pd.DataFrame(cost_values, index=shot_data.keys(), columns=['cost'])
df_x_text = pd.DataFrame(df_x_text_values, index=shot_data.keys(), columns=text_feature_index)
print df_x
print df_y
print df_x_text


['assets__alice', 'assets__backdrop', 'assets__banning', 'assets__billowy', 'assets__bunny', 'assets__bus', 'assets__car', 'assets__caterpillar', 'assets__clouds', 'assets__crow', 'assets__darcy', 'assets__deserted', 'assets__fern', 'assets__fields', 'assets__flash', 'assets__flower', 'assets__forest', 'assets__grass', 'assets__highway', 'assets__int', 'assets__island', 'assets__jimmy', 'assets__launcher', 'assets__leaf', 'assets__mansion', 'assets__matte', 'assets__morning', 'assets__mouse', 'assets__mrs', 'assets__police', 'assets__queen', 'assets__scare', 'assets__scary', 'assets__school', 'assets__shack', 'assets__sky', 'assets__squirrel', 'assets__stormy', 'assets__street', 'assets__submarine', 'assets__sunny', 'assets__tank', 'assets__tanker', 'assets__top', 'assets__trolley', 'cached_display_name', 'code', 'created_by', 'cut_duration', 'cut_in', 'cut_out', 'filmstrip_image', 'head_duration', 'head_in', 'head_out', 'open_notes_count', 'sg_cut_duration', 'sg_cut_in', 'sg_cut_order

In [2]:
# Set root package root directory.
import os
os.environ['ML_FOR_SG_ROOT'] = 'Z:\Dev\Github\ml-for-sg'

# Create shotgun data manager.
from lib.source_generator.shotgun.source_generator import ShotgunSourceGenerator
shotgun_source_generator = ShotgunSourceGenerator()

In [3]:
r = shotgun_source_generator.generate_source_data(project_id=70)

['assets__alice', 'assets__backdrop', 'assets__banning', 'assets__billowy', 'assets__bunny', 'assets__bus', 'assets__car', 'assets__caterpillar', 'assets__clouds', 'assets__crow', 'assets__darcy', 'assets__deserted', 'assets__fern', 'assets__fields', 'assets__flash', 'assets__flower', 'assets__forest', 'assets__grass', 'assets__highway', 'assets__int', 'assets__island', 'assets__jimmy', 'assets__launcher', 'assets__leaf', 'assets__mansion', 'assets__matte', 'assets__morning', 'assets__mouse', 'assets__mrs', 'assets__police', 'assets__queen', 'assets__scare', 'assets__scary', 'assets__school', 'assets__shack', 'assets__sky', 'assets__squirrel', 'assets__stormy', 'assets__street', 'assets__submarine', 'assets__sunny', 'assets__tank', 'assets__tanker', 'assets__top', 'assets__trolley', 'cached_display_name', 'code', 'created_by', 'cut_duration', 'cut_in', 'cut_out', 'filmstrip_image', 'head_duration', 'head_in', 'head_out', 'open_notes_count', 'sg_cut_duration', 'sg_cut_in', 'sg_cut_order

In [27]:
from IPython.display import display


import pandas as pd
r2=r['df_x_full']
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_rows", 500)
display(r['df_x_full'])
r2.to_csv(r'z:\Dev\pandas.csv', mode='w', sep=',')

,assets__alice,assets__backdrop,assets__banning,assets__billowy,assets__bunny,assets__bus,assets__car,assets__caterpillar,assets__clouds,assets__crow,...,tail_in,tail_out,task_template,tasks__animation,tasks__comp,tasks__fx,tasks__layout,tasks__light,type,updated_by
bunny_010_0010,False,True,True,True,True,False,True,False,True,True,...,None,None,None,True,True,True,True,True,Shot,Shotgun Support
bunny_010_0020,True,False,False,False,False,True,True,True,False,False,...,None,None,None,True,True,True,True,True,Shot,None
bunny_010_0030,False,False,True,False,False,False,True,True,False,False,...,None,None,None,True,True,True,True,True,Shot,None


In [17]:
new_dict = {'df_x_full': r['df_x_full'].to_json(), 'df_x_text_full': r['df_x_text_full'].to_json(), 'df_y_full': r['df_y_full'].to_json()}

In [18]:
new_dict

{'df_x_full': '{"assets__alice":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__backdrop":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__banning":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":1},"assets__billowy":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__bunny":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__bus":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__car":{"bunny_010_0010":1,"bunny_010_0020":1,"bunny_010_0030":1},"assets__caterpillar":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":1},"assets__clouds":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__crow":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__darcy":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__deserted":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__fern":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0

In [19]:
import json
json_data = json.dumps(new_dict)
f = open('z:\Dev\\test.json', 'w')
f.write(json_data)
f.close()

In [12]:
import numpy
def support_default(o):
    if isinstance(o,numpy.int64):
        o = int(o)
        return int(o)

In [28]:
f = open('z:\Dev\\test.json', 'r')
r2 = json.load(f)

In [32]:
r2['df_x_full']

u'{"assets__alice":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__backdrop":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__banning":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":1},"assets__billowy":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__bunny":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__bus":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__car":{"bunny_010_0010":1,"bunny_010_0020":1,"bunny_010_0030":1},"assets__caterpillar":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":1},"assets__clouds":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__crow":{"bunny_010_0010":1,"bunny_010_0020":0,"bunny_010_0030":0},"assets__darcy":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__deserted":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__fern":{"bunny_010_0010":0,"bunny_010_0020":1,"bunny_010_0030":0},"assets__fi

In [31]:
pd.DataFrame.from_dict(r2['df_x_full'])

ValueError: DataFrame constructor not properly called!

In [4]:
from lib.resource.resource_handler import ResourceHandler
rh = ResourceHandler()
rh.save_source_data(r)

In [32]:
import os
os.environ['ML_FOR_SG_ROOT'] = 'Z:\Dev\Github\ml-for-sg'
from lib.resource.resource_handler import ResourceHandler
rh = ResourceHandler()
r2 = rh.load_source_data()

In [33]:
r2['df_x_text_full']

,_i_,able,about,above,absurd,acceptance,accustomed,across,ada,added,...,yes,yesterday,yet,you,youd,your,yours,yourself,youth,zigzag
bunny_010_0010,0,1,1,0,0,0,0,0,0,1,...,1,0,0,1,0,1,1,0,0,0
bunny_010_0040,1,1,1,0,0,0,0,1,1,0,...,0,0,0,1,0,1,1,1,0,0
bunny_010_0050,0,1,1,0,0,0,0,1,1,0,...,1,1,1,1,0,1,0,0,1,0
bunny_010_0020,0,1,1,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
bunny_010_0030,0,1,1,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
bunny_010_0080,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,1,0,0,0,0
bunny_010_0090,0,0,1,0,0,0,1,0,0,1,...,1,0,1,1,1,1,0,0,1,0
bunny_010_0060,0,1,1,0,0,0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
bunny_010_0070,0,1,1,0,0,1,0,0,1,1,...,1,1,0,1,0,1,1,1,1,1
bunny_010_0100,0,1,1,1,1,0,0,0,0,1,...,1,1,1,1,0,1,1,1,0,0


In [34]:
import pandas as pd
df_x_full = r2['df_x_full']
df_y_full = r2['df_y_full']

In [43]:
df_x_full.fillna(0)

,assets__acorn,assets__alice,assets__all,assets__anders,assets__backdrop,assets__banning,assets__barn,assets__beach,assets__billowy,assets__blue,...,tail_in,tail_out,task_template,tasks__animation,tasks__comp,tasks__fx,tasks__layout,tasks__light,type,updated_by
bunny_010_0010,0,0,0,0,1,1,0,0,1,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,Shotgun Support
bunny_010_0040,1,0,1,0,0,0,0,0,0,1,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0050,0,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0020,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0030,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0080,0,0,1,0,1,0,0,1,0,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0090,0,0,1,0,0,0,0,0,1,1,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0060,0,0,1,0,0,0,1,0,0,1,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0070,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0
bunny_010_0100,0,0,1,1,0,0,0,0,1,0,...,0.0,0.0,0.0,1,1,1,1,1,Shot,0


In [48]:
import seaborn as sns
#for feature in df_x_full.columns:
#    t = pd.concat([df_x_full.filter(items=[feature]), df_y_full], axis=1)
#    print t.corr()
svm = sns.heatmap(df_x_full.fillna(0).corr())
svm.figure.savefig('z:\Dev\svm.png')

In [56]:
df_x_full.fillna(0).columns

Index([u'assets__acorn', u'assets__alice', u'assets__all', u'assets__anders',
       u'assets__backdrop', u'assets__banning', u'assets__barn',
       u'assets__beach', u'assets__billowy', u'assets__blue',
       ...
       u'tail_in', u'tail_out', u'task_template', u'tasks__animation',
       u'tasks__comp', u'tasks__fx', u'tasks__layout', u'tasks__light',
       u'type', u'updated_by'],
      dtype='object', length=144)

In [62]:
single = pd.concat([x.filter(items=['assets__acorn']), df_y_full], axis=1)
single.corr()
svm = sns.heatmap(single.corr())
svm.figure.savefig('z:\Dev\{}.png'.format(feature))

In [57]:
x = df_x_full.fillna(0)
for feature in x.columns[0]:
    single = pd.concat([x.filter(items=[feature]), df_y_full], axis=1)
    svm = sns.heatmap(single.corr())
    svm.figure.savefig('z:\Dev\{}.png'.format(feature))

KeyboardInterrupt: 